In [1]:
# ─── Celda 1: Imports y configuración de hilos ───
import os
# Reservamos 10 hilos de CPU para MKL/NumPy (dejando 2 libres)
os.environ["OMP_NUM_THREADS"] = "10"
os.environ["MKL_NUM_THREADS"] = "10"

import time
import numpy as np
import pandas as pd
import stumpy
from sklearn.preprocessing import StandardScaler

# Métrica Event-Wise F₀.₅
def event_wise_fbeta_score(y_true, y_pred, beta=0.5):
    def get_events(y):
        ev, in_ev = [], False
        for i,v in enumerate(y):
            if v and not in_ev:
                start, in_ev = i, True
            elif not v and in_ev:
                ev.append((start, i-1)); in_ev=False
        if in_ev: ev.append((start, len(y)-1))
        return ev
    true_ev = get_events(y_true)
    pred_ev = get_events(y_pred)
    TPe = sum(1 for a,b in true_ev if np.any(y_pred[a:b+1]==1))
    FNe = len(true_ev)-TPe
    FPe = sum(1 for a,b in pred_ev if not np.any(y_true[a:b+1]==1))
    FP_pts = np.sum((y_pred==1)&(y_true==0))
    N_nom   = np.sum(y_true==0)
    prec = TPe/(TPe+FPe+FP_pts/max(1,N_nom))
    rec  = TPe/max(1,TPe+FNe)
    b2   = beta**2
    return 0.0 if (prec+rec)==0 else (1+b2)*prec*rec/(b2*prec+rec)

# Parámetros
TRAIN_FILE = "data/train.parquet"
TEST_FILE  = "data/test.parquet"
starter_ch = [f"channel_{i}" for i in range(41,47)]
m = 100  # longitud de subsecuencia

In [2]:
# ─── Celda 2: Carga, imputación y escalado ───
# 1) Cargar únicamente las columnas necesarias
df      = pd.read_parquet(TRAIN_FILE, columns=["id","is_anomaly"] + starter_ch)
df_test = pd.read_parquet(TEST_FILE,  columns=["id"] + starter_ch)

# 2) Imputación de ceros con LOCF + BOCB, evitando inplace encadenado
for ch in starter_ch:
    tmp = df[ch].replace(0.0, np.nan)
    tmp = tmp.ffill().bfill()
    df[ch] = tmp

    tmp_t = df_test[ch].replace(0.0, np.nan)
    tmp_t = tmp_t.ffill().bfill()
    df_test[ch] = tmp_t

# 3) Escalado z-normalización a float64 (requisito de stumpy)
scaler = StandardScaler()
df[starter_ch]      = scaler.fit_transform(df[starter_ch]).astype(np.float64)
df_test[starter_ch] = scaler.transform(df_test[starter_ch]).astype(np.float64)

In [3]:
# ─── Celda 3: Cálculo EXACTO de MSTUMP multivariante ───
# Preparamos las series en float64
T_full = [df[ch].values.astype(np.float64) for ch in starter_ch]

start = time.time()
P_mat_full, I_full = stumpy.mstump(T_full, m)
elapsed = time.time() - start
print(f"MSTUMP exacto sobre {len(df):,} pts × {len(starter_ch)} canales: {elapsed/60:.1f} min")

KeyboardInterrupt: 

In [ ]:
# ─── Celda 4: Fusionar perfil y calibrar umbral ───
# 1) Fusionar multivariante
P_full = np.max(P_mat_full, axis=0)   # len = n - m + 1

# 2) Alinear etiquetas
y_full = df["is_anomaly"].to_numpy()   # len = n
y_trim = y_full[m-1:]                  # len = n - m + 1

# 3) Percentiles sobre nominales
mask_nom = (y_trim == 0)
lo, hi   = np.percentile(P_full[mask_nom], [90, 99.9])
thresholds = np.linspace(lo, hi, 100)

# 4) Búsqueda del mejor umbral
best_f05, best_thr = -1.0, lo
for thr in thresholds:
    preds = (P_full > thr).astype(int)
    f05   = event_wise_fbeta_score(y_trim, preds, beta=0.5)
    if np.isnan(f05): continue
    if f05 > best_f05:
        best_f05, best_thr = f05, thr

print(f"🏆 Umbral óptimo = {best_thr:.6f}, Event-Wise F₀.₅ = {best_f05:.4f}")

In [ ]:
# ─── Celda 5: Inferencia en test y creación de submission ───
# 1) Cálculo EXACTO en test (mstump multivariante)
T_test = [df_test[ch].values.astype(np.float64) for ch in starter_ch]
P_mat_test, _ = stumpy.mstump(T_test, m)
P_test = np.max(P_mat_test, axis=0)    # len = n_test - m + 1

# 2) Detección con el umbral
preds_trim = (P_test > best_thr).astype(int)
preds      = np.concatenate([np.zeros(m-1, dtype=int), preds_trim])

# 3) Guardar submission
submission = pd.DataFrame({
    "id":         df_test["id"].values,
    "is_anomaly": preds
})
submission.to_csv("submission_matrix_profile.csv", index=False)
print("▶ submission_matrix_profile.csv creada — anomalías detectadas:", preds.sum())

scrump(30%) sobre 14,728,321 pts x 6 canales en paralelo: 0.8 min
